In [ ]:
#connect drive for files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#enter the path to directory
%cd /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts

/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/nlp


In [ ]:
#install all requirements
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/Shareddrives/ECE%20226%20Project/archai-neurips-lts/archai-neurips-lts
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/NVIDIA/dllogger.git to /tmp/pip-install-xkw4r9p5/dllogger_df4ae39fdd394a149982cc82982339ce
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/dllogger.git /tmp/pip-install-xkw4r9p5/dllogger_df4ae39fdd394a149982cc82982339ce
  Resolved https://github.com/NVIDIA/dllogger.git to commit 0540a43971f4a8a16693a9de9de73c1072020769
  Preparing metadata (setup.py) ... done
  Attempting uninstall: archai
    Found existing installation: archai 0.6.0
    Uninstalling archai-0.6.0:
      Successfully uninstalled archai-0.6.0
  Running setup.py develop for archai


In [ ]:
#run search for gpt2 with param limits
!python archai/nlp/search.py --n_iter 30 --model_type hf_gpt2 --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp


baseline - finding the baseline architecture latency memory and decoder params

In [ ]:
!python archai/nlp/search.py --profile_baseline --model_type hf_gpt2 --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2

2023-06-13 05:48:07.577556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
STAGE:2023-06-13 05:50:01 2709:2709 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-13 05:50:01 2709:2709 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-13 05:50:01 2709:2709 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
config_0_j1 has: 
            330624 decoder_params
            2939776 total_params
            0.0030s latency
            58.2759MB memory
STAGE:2023-06-13 05:50:01 2709:2709 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-13 05:50:01 2709:2709 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-13 05:50:01 2709:2709 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
config_0_j2 has: 
            660736 decoder_params
            3269888 total_params
            0.0064s latency
            70.82

select pareto which are closest to the baseline

In [ ]:
!python archai/nlp/search.py --select_pareto --model_type hf_gpt2 --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2

2023-06-13 05:51:53.897504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected 18 of pareto jobs based on their proximity to the baseline architectures.
Unique Pareto-frontier jobs: 16


Training pareto architectures (will take time) change the path here. once trained, move the generated files in logdir to logdir/hf_gpt_titanxp_3d/pareto/

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j0  --n_layer 2 --d_model 128 --d_inner 256 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 05:57:55.839759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j0
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j0', experiment_name='config_0_j0', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j1  --n_layer 2 --d_model 256 --d_inner 768 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 06:06:48.398488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1', experiment_name='config_0_j1', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j2  --n_layer 2 --d_model 128 --d_inner 512 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 06:18:28.522823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2', experiment_name='config_0_j2', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j3  --n_layer 2 --d_model 128 --d_inner 256 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 06:27:16.344168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3', experiment_name='config_0_j3', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j4  --n_layer 2 --d_model 256 --d_inner 1920 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 06:35:55.426980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4', experiment_name='config_0_j4', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j5  --n_layer 2 --d_model 128 --d_inner 256 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 06:49:37.628824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5', experiment_name='config_0_j5', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j6  --n_layer 2 --d_model 192 --d_inner 1728 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 06:58:14.776966: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6', experiment_name='config_0_j6', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j7  --n_layer 2 --d_model 192 --d_inner 2176 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 07:09:56.089813: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7', experiment_name='config_0_j7', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j8  --n_layer 2 --d_model 384 --d_inner 832 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 07:22:14.673544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8', experiment_name='config_0_j8', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j9  --n_layer 2 --d_model 320 --d_inner 832 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 08:44:18.275381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j9
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j9', experiment_name='config_0_j9', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j10  --n_layer 2 --d_model 256 --d_inner 448 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 07:39:15.518394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j10
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j10', experiment_name='config_0_j10', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j11  --n_layer 2 --d_model 384 --d_inner 704 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 08:58:47.191980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j11
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j11', experiment_name='config_0_j11', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j12  --n_layer 2 --d_model 320 --d_inner 1088 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 08:18:35.241217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j12
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j12', experiment_name='config_0_j12', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j13  --n_layer 2 --d_model 128 --d_inner 2368 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 08:33:53.130246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j13
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j13', experiment_name='config_0_j13', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j14  --n_layer 2 --d_model 128 --d_inner 704 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 09:46:36.205113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j14
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j14', experiment_name='config_0_j14', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j15  --n_layer 2 --d_model 320 --d_inner 640 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 09:56:01.083785: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j15
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j15', experiment_name='config_0_j15', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j16  --n_layer 2 --d_model 128 --d_inner 448 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 10:10:16.445023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j16
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j16', experiment_name='config_0_j16', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j17  --n_layer 3 --d_model 320 --d_inner 640 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 10:18:55.117149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j17
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j17', experiment_name='config_0_j17', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

Training baselines ( will take time). Once training is done, move the files from logdir to baseline_logs/hf_gpt2/wt2/ folder

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j1  --n_layer 1 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 10:37:22.540953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1', experiment_name='config_0_j1', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j2  --n_layer 2 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 10:47:47.081495: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2', experiment_name='config_0_j2', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j3  --n_layer 3 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 10:59:02.790697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3', experiment_name='config_0_j3', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j4  --n_layer 4 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 11:10:44.751284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4', experiment_name='config_0_j4', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j5  --n_layer 5 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 11:23:21.740049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5', experiment_name='config_0_j5', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j6  --n_layer 6 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-13 11:36:52.763573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6', experiment_name='config_0_j6', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j7  --n_layer 7 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-14 15:47:41.328953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7', experiment_name='config_0_j7', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_gpt2 --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 8160 --vocab word --experiment_name config_0_j8  --n_layer 8 --d_model 256 --d_inner 128 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-14 16:03:52.724200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8', experiment_name='config_0_j8', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

Plot pareto baseline - compare the validation perplexities of baseline and LTS. results are generated in logdir/hf_gpt2_titanxp_3d/pareto

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/search.py --plot_pareto_baseline --n_layer 2 3 4 5 6 7 8 --div_val 4 --n_head 2 4 8 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2 --model_type hf_gpt2

2023-06-14 16:24:19.345513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
{'config_0_j1_train_log.json': {'train_elapsed': 609.9240190982819, 'valid_ppl': 156.49496743414474, 'amlt_job': None}, 'config_0_j2_train_log.json': {'train_elapsed': 658.4874751567841, 'valid_ppl': 134.7409846475925, 'amlt_job': None}, 'config_0_j3_train_log.json': {'train_elapsed': 686.098240852356, 'valid_ppl': 132.04010581413854, 'amlt_job': None}, 'config_0_j4_train_log.json': {'train_elapsed': 742.2006940841675, 'valid_ppl': 135.9099510112342, 'amlt_job': None}, 'config_0_j5_train_log.json': {'train_elapsed': 794.4957520961761, 'valid_ppl': 127.79329470207784, 'amlt_job': None}, 'config_0_j6_train_log.json': {'train_elapsed': 845.1245954036713, 'valid_ppl': 123.9567196875311, 'amlt_job': None}, 'config_0_j7_train_log.json': {'train_elapsed': 916.5193042755127, 'valid_ppl': 123.66564302704037, 'amlt_job': None}, 'config_0_j8_train_log.json': {'train_elapsed'